In [1]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Sequential, losses, optimizers
import pandas as pd
import numpy as np
from sklearn.metrics import roc_curve, auc, precision_recall_curve
import matplotlib.pyplot as plt

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
def One_hot_raw(path):
    seqs = open(path).readlines()

    X1 = [seq.split()[1] for seq in seqs if seq.strip() != '']
    y = [int(seq.split()[0]) for seq in seqs if seq.strip() != '']
    One = []
    for i in range(0, len(X1)):
        One.append(list(X1[i]))
    X = pd.DataFrame(One)
    return np.array(X), np.array(y)


def one_hot(rawDataFrame, codingMode=0):  # rawDataFrame is numpy.ndarray

    sampleSeq3DArr = rawDataFrame[:, :]

    if codingMode == 0:
        probMatr = convertSampleToProbMatr(sampleSeq3DArr)

    return probMatr


def convertSampleToProbMatr(sampleSeq3DArr):  # changed add one column for '1'

    letterDict = {"A": 0, "C": 1, "D": 2, "E": 3, "F": 4, "G": 5, "H": 6, "I": 7, "K": 8, "L": 9, "M": 10, "N": 11,
                  "P": 12, "Q": 13, "R": 14, "S": 15, "T": 16, "V": 17, "W": 18, "Y": 19, "X": 20}
    AACategoryLen = 21  # add -

    probMatr = np.zeros((len(sampleSeq3DArr), 1, len(sampleSeq3DArr[0]), AACategoryLen))

    sampleNo = 0
    for sequence in sampleSeq3DArr:

        AANo = 0
        for AA in sequence:

            if not AA in letterDict:
                probMatr[sampleNo][0][AANo] = np.full((1, AACategoryLen), 1.0 / AACategoryLen)

            else:
                index = letterDict[AA]
                probMatr[sampleNo][0][AANo][index] = 1

            AANo += 1
        sampleNo += 1

    return probMatr

In [4]:
def build_network():
    # 先创建包含多网络层的列表
    conv_layers = [
        layers.Conv1D(filters=128, kernel_size=1, padding='same', activation=tf.nn.relu),
        layers.Dropout(0.5),
        layers.MaxPooling1D(2),
        layers.Conv1D(filters=128, kernel_size=3, padding='same', activation=tf.nn.relu),
        layers.Dropout(0.5),
        layers.MaxPooling1D(2),
        layers.Conv1D(filters=128, kernel_size=9, padding='same', activation=tf.nn.relu),
        layers.MaxPooling1D(2),
        layers.Dropout(0.5)
    ]

    fc_layers = [
        layers.Dense(64, activation=tf.nn.relu),  # 全连接层， 64 个节点
        layers.GlobalAveragePooling1D(),
        layers.Dense(1, activation=tf.nn.sigmoid)  # 全连接层， 1 个节点
    ]

    conv_layers.extend(fc_layers)
    network = Sequential(conv_layers)
    network.build(input_shape=[None, 41, 21])#[None, 41, 21]
    base_learning_rate = 0.001
    network.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate), loss='binary_crossentropy',metrics=['accuracy'])#编译模型
    #network.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    network.summary()
    return network

In [5]:
def save_predict_result(data, output):
    with open(output, 'w') as f:
        f.write('# result for true and predict \n')
        for i in data:
            f.write('%d\t%.8f\n' % (i[0], float(i[1])))
    return None

In [6]:
def preprocess(x, y):
    x = tf.cast(x, dtype=tf.float32)
    y = tf.cast(y, dtype=tf.int32)
    return x, y

In [7]:
def evaluate(X, y, indep=None, batch_size=256, epochs=150):
    classes = sorted(list(set(y)))

    X_train, y_train = X, y
    X_test, y_test = indep[0], indep[1]

    X_train = one_hot(X_train)
    X_test = one_hot(X_test)

    X_train_t = X_train
    X_test_t = X_test
    X_train_t.shape = (X_train_t.shape[0], X_train_t.shape[2], X_train_t.shape[3])
    X_test_t.shape = (X_test_t.shape[0], X_test_t.shape[2], X_test_t.shape[3]) #删除无效维度
    # 构建训练集对象，随机打乱，预处理，批量化
    train_db = tf.data.Dataset.from_tensor_slices((X_train_t, y_train))
    train_db = train_db.shuffle(len(X)).map(preprocess).batch(batch_size)
    # 构建测试集对象，预处理，批量化
    test_db = tf.data.Dataset.from_tensor_slices((X_test_t, y_test))
    test_db = test_db.map(preprocess).batch(batch_size)

    network = build_network()
    best_saving = tf.keras.callbacks.ModelCheckpoint(filepath=r'D:\PycharmProjects\pythonProject\Papernew_datachange\model\based_model\5k\UbidelAcet_CNN_OH\dataoh.h5', monitor='val_loss',verbose=1, save_best_only=True, save_weights_only=False)
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)
    history = network.fit(train_db, epochs=epochs, validation_data=test_db, verbose=2,callbacks=[best_saving, early_stopping], batch_size=batch_size)

    print("Independent test:", network.evaluate(test_db))
    tmp_result = np.zeros((len(y_test), len(classes)))
    tmp_result[:, 0], tmp_result[:, 1] = y_test, network.predict(X_test_t, batch_size=batch_size)[:, 0]
    return tmp_result, history, y_test

In [8]:
def main():
    os.chdir(r'D:\PycharmProjects\pythonProject\Papernew_datachange\data\based\5k\Acet_Ubi\UbidelAcet')
    epoch = 300
    X, y = One_hot_raw('train.txt')
    os.chdir(r'D:\PycharmProjects\pythonProject\Papernew_datachange\data\based\NEDD')
    indep = One_hot_raw('test.txt')
    os.chdir(r'D:\PycharmProjects\pythonProject\Papernew_datachange\model\based_model\5k\UbidelAcet_CNN_OH')
    ind_res, history, y_test = evaluate(X, y, indep=indep, epochs=epoch, batch_size=256)
    save_predict_result(ind_res, r'D:\PycharmProjects\pythonProject\Papernew_datachange\model\based_model\5k\UbidelAcet_CNN_OH\dataoh.txt')
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    plt.subplot(1, 2, 1)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.savefig('new-data-cnn.png')
    
if __name__ == '__main__':
    main()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 41, 128)           2816      
_________________________________________________________________
dropout (Dropout)            (None, 41, 128)           0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 20, 128)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 20, 128)           49280     
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 10, 128)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 10, 128)           1

45/45 - 0s - loss: 0.6186 - accuracy: 0.6554 - val_loss: 0.6494 - val_accuracy: 0.6258

Epoch 00031: val_loss improved from 0.65423 to 0.64939, saving model to D:\PycharmProjects\pythonProject\Papernew_datachange\model\based_model\5k\UbidelAcet_CNN_OH\dataoh.h5
Epoch 32/300
45/45 - 0s - loss: 0.6191 - accuracy: 0.6539 - val_loss: 0.6625 - val_accuracy: 0.6108

Epoch 00032: val_loss did not improve from 0.64939
Epoch 33/300
45/45 - 0s - loss: 0.6152 - accuracy: 0.6628 - val_loss: 0.6484 - val_accuracy: 0.6068

Epoch 00033: val_loss improved from 0.64939 to 0.64835, saving model to D:\PycharmProjects\pythonProject\Papernew_datachange\model\based_model\5k\UbidelAcet_CNN_OH\dataoh.h5
Epoch 34/300
45/45 - 0s - loss: 0.6125 - accuracy: 0.6598 - val_loss: 0.6429 - val_accuracy: 0.6307

Epoch 00034: val_loss improved from 0.64835 to 0.64290, saving model to D:\PycharmProjects\pythonProject\Papernew_datachange\model\based_model\5k\UbidelAcet_CNN_OH\dataoh.h5
Epoch 35/300
45/45 - 0s - loss: 0.60

45/45 - 0s - loss: 0.5302 - accuracy: 0.7320 - val_loss: 0.6475 - val_accuracy: 0.6096

Epoch 00081: val_loss did not improve from 0.62838
Epoch 82/300
45/45 - 0s - loss: 0.5348 - accuracy: 0.7254 - val_loss: 0.6286 - val_accuracy: 0.6448

Epoch 00082: val_loss did not improve from 0.62838
Epoch 83/300
45/45 - 0s - loss: 0.5353 - accuracy: 0.7316 - val_loss: 0.6259 - val_accuracy: 0.6416

Epoch 00083: val_loss improved from 0.62838 to 0.62589, saving model to D:\PycharmProjects\pythonProject\Papernew_datachange\model\based_model\5k\UbidelAcet_CNN_OH\dataoh.h5
Epoch 84/300
45/45 - 0s - loss: 0.5281 - accuracy: 0.7334 - val_loss: 0.6668 - val_accuracy: 0.5922

Epoch 00084: val_loss did not improve from 0.62589
Epoch 85/300
45/45 - 0s - loss: 0.5302 - accuracy: 0.7306 - val_loss: 0.6631 - val_accuracy: 0.5979

Epoch 00085: val_loss did not improve from 0.62589
Epoch 86/300
45/45 - 0s - loss: 0.5242 - accuracy: 0.7312 - val_loss: 0.6457 - val_accuracy: 0.6104

Epoch 00086: val_loss did not

10/10 [==============================] - 0s 4ms/step - loss: 0.6466 - accuracy: 0.6226
Independent test: [0.6465590596199036, 0.6225728392601013]


In [9]:
f = open(r'D:\PycharmProjects\pythonProject\Papernew_datachange\model\based_model\5k\UbidelAcet_CNN_OH\dataoh.txt')
y_true = []
pred = []
for line in f:
    if line.startswith('#'):
        pass
    else:
        name = line.replace('\n','').split('	')
        y_true.append(int(name[0]))
        pred.append(float(name[1]))
f.close()

In [10]:
# 性能评估
def calculate_metrics(labels, scores, cutoff=0.5, po_label=1):
    my_metrics = {
        'SN': 'NA',
        'SP': 'NA',
        'ACC': 'NA',
        'MCC': 'NA',
        'Recall': 'NA',
        'Precision': 'NA',
        'F1-score': 'NA',
        'Cutoff': cutoff,
    }

    tp, tn, fp, fn = 0, 0, 0, 0
    for i in range(len(scores)):
        if labels[i] == po_label:
            if scores[i] >= cutoff:
                tp = tp + 1
            else:
                fn = fn + 1
        else:
            if scores[i] < cutoff:
                tn = tn + 1
            else:
                fp = fp + 1

    my_metrics['SN'] = tp / (tp + fn) if (tp + fn) != 0 else 'NA'
    my_metrics['SP'] = tn / (fp + tn) if (fp + tn) != 0 else 'NA'
    my_metrics['ACC'] = (tp + tn) / (tp + fn + tn + fp)
    my_metrics['MCC'] = (tp * tn - fp * fn) / np.math.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)) if (
                                                                                                                     tp + fp) * (
                                                                                                                     tp + fn) * (
                                                                                                                     tn + fp) * (
                                                                                                                     tn + fn) != 0 else 'NA'
    my_metrics['Precision'] = tp / (tp + fp) if (tp + fp) != 0 else 'NA'
    my_metrics['Recall'] = my_metrics['SN']
    my_metrics['F1-score'] = 2 * tp / (2 * tp + fp + fn) if (2 * tp + fp + fn) != 0 else 'NA'
    return my_metrics

In [11]:
metric = calculate_metrics(y_true, pred)
metric

{'SN': 0.5501618122977346,
 'SP': 0.6949838187702265,
 'ACC': 0.6225728155339806,
 'MCC': 0.24775755978729957,
 'Recall': 0.5501618122977346,
 'Precision': 0.6433301797540208,
 'F1-score': 0.5931094635848234,
 'Cutoff': 0.5}